In [1]:
from qdrant_client import QdrantClient, models

/Users/tacofoil/Desktop/Education/Data Talks Club/LLM/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Q1

In [5]:
from fastembed import TextEmbedding
import numpy as np

model = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")
query = "I just discovered the course. Can I join now?"
embedding = next(model.embed([query]))

embedding_array = np.array(embedding)
print("Shape:", embedding_array.shape)
print("Minimum value:", embedding_array.min())


Shape: (512,)
Minimum value: -0.11726373551188797


# Q2

In [10]:
np.linalg.norm(embedding_array)

np.float64(1.0)

In [14]:
doc = 'Can I still join the course after the start date?'

In [15]:
embedding_doc = next(model.embed([doc]))
embedding_array_doc = np.array(embedding_doc)

In [16]:
embedding_array.dot(embedding_array_doc)

np.float64(0.9008528856818037)

# Q3

In [17]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [ ]:
docs = [d["text"] for d in documents]
doc_vectors = list(model.embed(docs))
doc_vectors = np.array(doc_vectors)

[array([-2.49524888e-02, -3.96454003e-02, -4.37674569e-03,  2.95830151e-02,
        -1.20300759e-02, -6.93250284e-03, -7.65633787e-05,  6.15121306e-02,
        -2.85105810e-02, -6.70960999e-02,  6.53191987e-03, -5.09575828e-02,
        -5.01244122e-02, -5.83276288e-02,  9.54578951e-02, -5.41039744e-02,
        -3.20385997e-02, -4.69560360e-02, -6.16867892e-02, -9.03934413e-02,
        -3.35349292e-02, -2.90533885e-02,  4.44972100e-02, -7.00045630e-02,
         4.27595350e-02, -7.01989179e-02, -8.68037299e-02,  8.14205920e-02,
         7.21879766e-02,  1.89472973e-02, -5.95576924e-02,  1.09493171e-02,
        -1.80511222e-02,  4.60817713e-02, -2.97588590e-02,  3.45792134e-02,
         4.05221465e-02,  1.75949984e-02, -2.77041562e-02, -1.67211612e-02,
        -5.02860049e-02,  1.34971681e-02,  9.17410166e-02,  8.32859014e-03,
        -7.49552597e-02,  2.35090027e-03, -4.66536598e-02,  1.29560662e-02,
         2.57738761e-02,  5.87155689e-03, -4.75062520e-02,  1.41338270e-02,
        -1.0

In [24]:
# Get vector withb highest similarity
V = np.vstack(doc_vectors)  # shape: (N_docs, 512)

In [23]:
# Get vector withb highest similarity
for i, doc in enumerate(documents):
    similarity = embedding_array.dot(V[i])
    print(f"Similarity with doc {i}: {similarity:.4f} - {doc['question']}")
    documents[i]["similarity"] = similarity


Similarity with doc 0: 0.7630 - Course - Can I still join the course after the start date?
Similarity with doc 1: 0.8182 - Course - Can I follow the course after it finishes?
Similarity with doc 2: 0.8085 - Course - When will the course start?
Similarity with doc 3: 0.7133 - Course - What can I do before the course starts?
Similarity with doc 4: 0.7304 - How can we contribute to the course?


# Q4

In [26]:
full_text = doc['question'] + ' ' + doc['text']
full_text

'How can we contribute to the course? Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.'

In [29]:
# Test the similarity with a new field (concatentation of question and text)
full_texts = [doc['question'] + ' ' + doc['text'] for doc in documents]
full_vectors = list(model.embed(full_texts))  # each is a 512-dim vector

In [30]:
# Compute similarity between query and each full_text embedding
for i, (doc, vector) in enumerate(zip(documents, full_vectors)):
    similarity = embedding_array.dot(vector)
    print(f"Similarity with doc {i}: {similarity:.4f} - {doc['question']}")
    doc["similarity_q4"] = similarity  # store under a different key if you want to compare with Q3

Similarity with doc 0: 0.8515 - Course - Can I still join the course after the start date?
Similarity with doc 1: 0.8437 - Course - Can I follow the course after it finishes?
Similarity with doc 2: 0.8408 - Course - When will the course start?
Similarity with doc 3: 0.7755 - Course - What can I do before the course starts?
Similarity with doc 4: 0.8086 - How can we contribute to the course?


# Q5

In [37]:
import json

for model in TextEmbedding.list_supported_models():
    print(json.dumps(model, indent=2))

{
  "model": "BAAI/bge-base-en",
  "sources": {
    "hf": "Qdrant/fast-bge-base-en",
    "url": "https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.42,
  "additional_files": [],
  "dim": 768,
  "tasks": {}
}
{
  "model": "BAAI/bge-base-en-v1.5",
  "sources": {
    "hf": "qdrant/bge-base-en-v1.5-onnx-q",
    "url": "https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en-v1.5.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.21,
  "additional_files": [],
  "dim": 768,
  

In [ ]:
# Get the list of supported models
supported_models = TextEmbedding.list_supported_models()

# Extract all dimensions
dimensions = [model["dim"] for model in supported_models]

# Get the smallest dimension
min_dim = min(dimensions)
print("Smallest embedding dimension:", min_dim)


Smallest embedding dimension: 384


# Q6

In [ ]:
model = TextEmbedding(model_name="BAAI/bge-small-en")

In [38]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [36]:
documents

[{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
  'section': 'General course-related questions',
  'question': 'How do I sign up?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'The course videos are pre-recorded, you can start watching the course right now.\nWe will also occasionally have office hours - live sessions where we will answer your questions. The office hours sessions are recorded too.\nYou can see the office hours as well as the pre-recorded course videos in the course playlist on YouTube.',
  'section': 'General course-related questions',
  'question': 'Is it going to be live? When?',
  'course': 'machi

In [39]:
for doc in documents:
    doc["text"] = doc["question"] + " " + doc["text"]


In [44]:
client = QdrantClient("http://localhost:6333")  # or QdrantCloud if you use it
embedding_model = TextEmbedding(model_name="BAAI/bge-small-en")
collection_name = "ml-zoomcamp-faq-384"

client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=384,
        distance=models.Distance.COSINE
    )
)

True

In [45]:
points = []
for i, doc in enumerate(documents):
    embedding = next(embedding_model.embed([doc["text"]]))

    point = models.PointStruct(
        id=i,
        vector=embedding,
        payload={
            "question": doc["question"],
            "answer": doc["text"],
            "course": doc["course"]
        }
    )
    points.append(point)

client.upsert(collection_name=collection_name, points=points)


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [46]:
query = "I just discovered the course. Can I join now?"
query_vector = next(embedding_model.embed([query]))

results = client.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=1,
    with_payload=True
)

top_score = results[0].score
print(f"Top score: {top_score}")


Top score: 0.8703172


/var/folders/55/_k7nlpxj5991v2tpz20h4gfw0000gn/T/ipykernel_59334/2706538688.py:4: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(
